In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Initialising the test and training set

(x_train, labels_train), (x_test, labels_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
y_train = keras.utils.to_categorical(labels_train, 10)
y_test = keras.utils.to_categorical(labels_test, 10)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

# define data preparation
datagen = ImageDataGenerator(rotation_range=10,zoom_range = 0.10,width_shift_range=0.1,height_shift_range=0.1)
# fit parameters from data
datagen.fit(x_train)
# configure batch size and retrieve one batch of images
X_batch = datagen.flow(x_train, labels_train, batch_size=100)

b_size=100
c_counter=0
X_new=[]
Y_new=[]
for i in range(len(X_batch)):
  X_new.append(X_batch[i][0].flatten())
  c_counter+=1
  Y_new.append(X_batch[i][1].flatten())

Y_new=(np.array(Y_new).flatten()).reshape(c_counter*b_size,)
X_new=np.array(X_new).reshape(c_counter*b_size,28,28,1)
Y_new_test = keras.utils.to_categorical(Y_new, 10)


x_train_new=np.concatenate((x_train, X_new), axis=0)
y_train_new=np.concatenate((y_train, Y_new_test), axis=0)
labels_train_new=np.concatenate((labels_train, Y_new), axis=0)




In [ ]:
#Helper functions
import pandas as pd

#Get the error rates per digits
def error_rate_per_digit(labels_predicted,labels_test):
    c=[]
    d=[]
    o_c=[]
    o_d=[]
    for i in range(len(labels_predicted)):
        o_c.append(labels_test[i])
        o_d.append(1)
        if labels_predicted[i] != labels_test[i]:
            c.append(labels_test[i])
            d.append(1)
    e=pd.DataFrame(np.transpose([c]))
    f=pd.DataFrame(np.transpose([d]))
    g=e.merge(f,left_index=True,right_index=True)
    o_e=pd.DataFrame(np.transpose([o_c]))
    o_f=pd.DataFrame(np.transpose([o_d]))
    o_g=o_e.merge(o_f,left_index=True,right_index=True)
    final=o_g.groupby('0_x').count().reset_index().merge(g.groupby('0_x').count().reset_index(),left_index=True,right_index=True)
    final=final.rename(columns={"0_x_x":'digits',"0_y_x":"Predicted_Correct",'0_y_y':'Predcited_incorrectly'})
    final['error_rate']=np.round(final['Predcited_incorrectly']/final['Predicted_Correct'],4)*100
    del final['0_x_y']
    return final
  #  get_error_rate=error_rate_per_digit(labels_predicted,labels_test)
  #  print(get_error_rate)

#Shows the digits in a plot
import matplotlib.pyplot as plt
def show_digits(x_test,labels_predicted,misclassified):
  fig_size_est=round(misclassified/10)
  plt.figure(figsize=(10,10))
  c_counter=1
  ax_counter=1
  for i in range(0,100):
      for j in range(0,100):
          if labels_predicted[c_counter-1] != labels_test[c_counter-1]:
              #print(labels_predicted[c_counter-1],labels_test[c_counter-1],'\n')
              ax=plt.subplot(10,10,ax_counter)
              plt.imshow(x_test[c_counter-1,:].reshape(28,28), cmap=plt.get_cmap('gray_r'))
              plt.title(labels_predicted[c_counter-1])
              ax.get_xaxis().set_visible(False)
              ax.get_yaxis().set_visible(False)
              ax_counter+=1
          c_counter+=1

def misclassification_error(net,x_test,labels_test):
  outputs=net.predict(x_test)
  labels_predicted=np.argmax(outputs, axis=1)
  misclassified=sum(labels_predicted!=labels_test)
  print('Percentage misclassified = ',100*misclassified/labels_test.size)
  return labels_predicted


In [ ]:
#---------------------------------------------------

In [ ]:
#Model Information here
net = Sequential()
net.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu',input_shape=(28,28,1)))
net.add(BatchNormalization())
net.add(MaxPool2D(pool_size=(2, 2)))
net.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
net.add(BatchNormalization())
net.add(Dropout(rate=0.5))
net.add(Conv2D(32, (3, 3), activation='relu',padding='same'))
net.add(BatchNormalization())
net.add(Dropout(rate=0.5))
net.add(Conv2D(64, (3, 3), activation='relu'))
net.add(MaxPool2D(pool_size=(2, 2)))
net.add(BatchNormalization())
net.add(Conv2D(128, (3, 3), activation='relu'))
net.add(MaxPool2D(pool_size=(2, 2)))
net.add(BatchNormalization())
net.add(Flatten())
net.add(Dropout(rate=0.5))
net.add(Dense(256, activation='relu'))
net.add(Dropout(rate=0.5))
net.add(Dense(512, activation='relu'))
net.add(Dropout(rate=0.5))
net.add(Dense(1024, activation='relu'))
net.add(Dropout(rate=0.5))
net.add(Dense(512, activation='relu'))
net.add(Dropout(rate=0.5))
net.add(Dense(256, activation='relu'))
net.add(Dropout(rate=0.5))
net.add(Dense(10, activation='softmax'))
net.summary()
net.compile(loss='categorical_crossentropy', optimizer='adam')


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
batch_normalization_10 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 32)        9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 12, 12, 32)        128       
_________________________________________________________________
dropout_16 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 12, 12, 32)       

In [ ]:
#----------------------------------------------------------------

In [ ]:
#Loading Weights or Model if needed
net.load_weights("network_for_mnist_weights.h5")

In [ ]:
#Training the model here
#history = net.fit(x_train,y_train,validation_data=(x_train_new, y_train_new), epochs=50, batch_size=16)
history = net.fit(datagen.flow(x_train, y_train, batch_size=256),steps_per_epoch=len(x_train) / 256, epochs=20)
net.save_weights('network_for_mnist_weights.h5')

Epoch 1/20
234/234 [==============================] - 14s 58ms/step - loss: 0.0483
Epoch 2/20
234/234 [==============================] - 14s 58ms/step - loss: 0.0486
Epoch 3/20
234/234 [==============================] - 14s 59ms/step - loss: 0.0444
Epoch 4/20
234/234 [==============================] - 14s 59ms/step - loss: 0.0444
Epoch 5/20
234/234 [==============================] - 14s 59ms/step - loss: 0.0478
Epoch 6/20
234/234 [==============================] - 14s 59ms/step - loss: 0.0468
Epoch 7/20
234/234 [==============================] - 14s 59ms/step - loss: 0.0430
Epoch 8/20
234/234 [==============================] - 14s 58ms/step - loss: 0.0475
Epoch 9/20
234/234 [==============================] - 14s 60ms/step - loss: 0.0479
Epoch 10/20
234/234 [==============================] - 14s 58ms/step - loss: 0.0550
Epoch 11/20
234/234 [==============================] - 14s 58ms/step - loss: 0.0526
Epoch 12/20
234/234 [==============================] - 14s 59ms/step - loss: 0.0482
E

In [ ]:
#Testing the model on the MNIST test set
labels_predicted=misclassification_error(net,x_test,labels_test)
get_error_rate=error_rate_per_digit(labels_predicted,labels_test)
print(get_error_rate)

Percentage misclassified =  0.37
   digits  Predicted_Correct  Predcited_incorrectly  error_rate
0       0                980                      2        0.20
1       1               1135                      8        0.70
2       2               1032                      3        0.29
3       3               1010                      2        0.20
4       4                982                      2        0.20
5       5                892                      5        0.56
6       6                958                      2        0.21
7       7               1028                      3        0.29
8       8                974                      4        0.41
9       9               1009                      6        0.59


In [ ]:
pip install emnist

In [ ]:
#Validation on the EMNIST Training data
from emnist import extract_training_samples
emnist_image, emnist_labels = extract_training_samples('digits')
emnist_image = emnist_image.reshape(240000, 784)
emnist_image = emnist_image.astype('float32')
emnist_image /= 255
emnist_labels_1 = keras.utils.to_categorical(emnist_labels, 10)
emnist_image = emnist_image.reshape(emnist_image.shape[0], 28, 28, 1)

#outputs=net2.predict(emnist_image)
#labels_predicted=np.argmax(outputs, axis=1)
#misclassified=sum(labels_predicted!=emnist_labels)
#print('Percentage misclassified = ',100*misclassified/emnist_labels.size)

labels_predicted_emnist_train=misclassification_error(net,emnist_image,emnist_labels)
get_error_rate=error_rate_per_digit(labels_predicted_emnist_train,emnist_labels)
print(get_error_rate)

Percentage misclassified =  0.44458333333333333
   digits  Predicted_Correct  Predcited_incorrectly  error_rate
0       0              24000                    109        0.45
1       1              24000                    102        0.42
2       2              24000                    105        0.44
3       3              24000                    109        0.45
4       4              24000                    100        0.42
5       5              24000                    113        0.47
6       6              24000                     52        0.22
7       7              24000                     38        0.16
8       8              24000                    120        0.50
9       9              24000                    219        0.91


In [ ]:
#Testing on the EMNIST Test Data
from emnist import extract_test_samples
emnist_image_test, emnist_labels_test = extract_test_samples('digits')
emnist_image_test = emnist_image_test.reshape(40000, 784)
emnist_image_test = emnist_image_test.astype('float32')
emnist_image_test /= 255
emnist_labels_1_test = keras.utils.to_categorical(emnist_labels_test, 10)
emnist_image_test = emnist_image_test.reshape(emnist_image_test.shape[0], 28, 28, 1)

#outputs=net.predict(emnist_image_test)
#labels_predicted_emnist=np.argmax(outputs, axis=1)
#misclassified=sum(labels_predicted_emnist!=emnist_labels_test)
#print('Percentage misclassified = ',100*misclassified/emnist_labels_test.size)
labels_predicted_emnist=misclassification_error(net,emnist_image_test,emnist_labels_test)
get_error_rate=error_rate_per_digit(labels_predicted_emnist,emnist_labels_test)
print(get_error_rate)

Percentage misclassified =  0.485
   digits  Predicted_Correct  Predcited_incorrectly  error_rate
0       0               4000                     34        0.85
1       1               4000                     22        0.55
2       2               4000                     13        0.32
3       3               4000                     19        0.48
4       4               4000                     24        0.60
5       5               4000                     18        0.45
6       6               4000                      7        0.18
7       7               4000                      4        0.10
8       8               4000                     10        0.25
9       9               4000                     43        1.07


In [ ]:
#Testing on the Augmented Data
labels_predicted=misclassification_error(net,X_new,Y_new)
get_error_rate=error_rate_per_digit(labels_predicted,Y_new)
print(get_error_rate)

Percentage misclassified =  0.3566666666666667
   digits  Predicted_Correct  Predcited_incorrectly  error_rate
0       0               5923                     10        0.17
1       1               6742                     22        0.33
2       2               5958                     10        0.17
3       3               6131                     24        0.39
4       4               5842                     34        0.58
5       5               5421                     37        0.68
6       6               5918                     12        0.20
7       7               6265                     15        0.24
8       8               5851                     16        0.27
9       9               5949                     34        0.57


In [ ]:
#If all looks good saving the Model weights and the model
net.save_weights('network_for_mnist_weights_Prateek_1.h5')
#net.save("network_for_mnist_latest.h5")
#from keras.models import load_model
#net=load_model("network_for_mnist_weights_Prateek.h5")

In [ ]:
net.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_50 (Conv2D)           (None, 24, 24, 32)        832       
_________________________________________________________________
batch_normalization_49 (Batc (None, 24, 24, 32)        128       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 12, 12, 32)        9248      
_________________________________________________________________
batch_normalization_50 (Batc (None, 12, 12, 32)        128       
_________________________________________________________________
dropout_67 (Dropout)         (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_52 (Conv2D)           (None, 12, 12, 32)      

In [ ]:
net.save_weights('network_for_mnist_weights.h5')
net.save("network_for_mnist.h5")